In [ ]:
!pip -q install langchain==0.0.150 openai huggingface_hub
!pip -q install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.4/648.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Setting up some keys

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.150
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tqdm
Required-by: 


In [ ]:
pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



# Custom Tools & Agents 🤖

In [ ]:
#@title Keys

import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [ ]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
from langchain import HuggingFaceHub

from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
model=HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0, "max_length":64})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

## Standard Tool

In [ ]:
from langchain.tools import DuckDuckGoSearchTool
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchTool()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

## Custom tools

In [ ]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [ ]:

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

## Random number

In [ ]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [ ]:
random_num()

2

In [ ]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

## Creating an agent

In [ ]:
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [ ]:
conversational_agent("What time is it in London?")



> Entering new AgentExecutor chain...


JSONDecodeError: ignored

In [ ]:

conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...


JSONDecodeError: ignored

In [ ]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The meaning of life is a philosophical question that has been debated by scholars, theologians, and philosophers for centuries. There is no one definitive answer to this question, as it is largely a matter of personal belief and interpretation. Some people believe that the meaning of life is to seek happiness, while others believe that it is to fulfill a specific purpose or destiny. Ultimately, the meaning of life is a deeply personal and subjective question that each individual must answer for themselves."
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}),
  AIMessage(content='The current local time in London is 59 minutes ahead of apparent solar time. Sunrise: 05:52AM. Sunset: 08:06PM. Day length: 14h 14m. Solar noon: 12:59PM. London uses Greenwich Mean Time (GMT) during standard time and British Summer Time (BST) during Daylight Saving Time (DST), or summer time.', additional_kwargs={}),
  HumanMessage(content='Can you give me a random number?', additional_kwargs={}),
  AIMessage(content='The response to your last comment was a random number, which was 4.', additional_kwargs={})],
 'output': 'The meaning of life is a philosophical question that has been debated by scholars, theologians, and philosophers for centuries. There is no one definitive answer to this question, as it is largely a matter of personal belief and interpretation. Some people believe that the meaning of life is to seek happiness, while

In [ ]:
# system prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range 

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

In [ ]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [ ]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...


JSONDecodeError: ignored

## Using the Tool Class - lets make a useful tool


In [ ]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content


In [ ]:
stripped_webpage('https://www.google.com')


'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchWatch the NBA Finals live on YouTube TVAdvertisingBusiness SolutionsAbout Google© 2023 - Privacy - Terms  '

In [ ]:
# webpage_tool = Tool(
#     name='Get Webpage',
#     func= stripped_webpage,
#     description="Useful for when you need to get the http from a specific webpage"
# )


In [ ]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content)
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content

    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

## Init the Agent again

In [ ]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [ ]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of to

In [ ]:
conversational_agent.run("Is there an article about Clubhouse on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/search/clubhouse"
}
Observation: 










 


You searched for clubhouse | TechCrunch




















































 


 





 







TechCrunch


















plus-bold














TechCrunch


Open Navigation






TechCrunch















Load More










 
 



Thought:

JSONDecodeError: ignored

In [ ]:
conversational_agent.run("What are the titles of the top stories on www.cbsnews.com/?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
```
Observation: 
CBS News - Breaking news, 24/7 live streaming news & top stories













Montana Rep. Zooey Zephyr




Disney vs. DeSantis




Chernobyl Anniversary




Freddie Mercury Auction




CBS News Live




Managing Your Money




Essentials Shopping




Newsletters


















News




US




World




Politics




Entertainment




HealthWatch




MoneyWatch




CBS Village




Technology




Science




Crime




Sports




Essentials








Pence appears for 7 hours before grand jury




Proliferation of modified weapons cause for alarm, officials say




Two Army helicopters collide and crash in Alaska, killing three soldiers




E. Jean Carroll said she kept quiet about rape accusation out of "fear" of Trump




Democrats see abortion as winning issue with 2024 Biden reelection launch




Brittney Griner gives first press conference

'Here are the titles of the top stories on CBS News:\n\n- Pence appears for 7 hours before grand jury\n- Proliferation of modified weapons cause for alarm, officials say\n- Two Army helicopters collide and crash in Alaska, killing three soldiers\n- E. Jean Carroll said she kept quiet about rape accusation out of "fear" of Trump\n- Democrats see abortion as winning issue with 2024 Biden reelection launch\n- Brittney Griner gives first press conference since Russian detainment\n- Trans, nonbinary state lawmakers press Biden on students athletics\n- Teens accused in Colorado rock-throwing death took "memento" photo of car\n- Lamar Jackson signs $260 million deal with Baltimore Ravens'